设置仿真参数

In [32]:
import random
import simpy

初始化仿真对象

In [33]:
RANDOM_SEED = 42
REQ_TOTAL = 100 # 总请求数
REQ_IAT   = 8.0 # 请求平均到达时间
SVR_ST    = 7.0 # 平均服务时间
MAX_WAIT  = 3.0 # 等待超时门限

latency = [] # 采集请求延迟

def request(env, name, server, service_time):
    arrive = env.now
    print('%7.4f %s: incoming' % (arrive, name))

    with server.request() as req:
        # 等待服务或超时
        results = yield req | env.timeout(MAX_WAIT)
        wait = env.now - arrive

    if req in results:
        # 开始服务
        print('%7.4f %s: waited %6.3f' % (env.now, name, wait))
        t = random.expovariate(1.0 / service_time)
        yield env.timeout(t)
        elapsed = env.now - arrive
        latency.append(elapsed) # 正常完成的请求，采集延迟
        print('%7.4f %s: completed %6.3f' % (env.now, name, elapsed))
    else:
        # 超时
        print('%7.4f %s: time out after %6.3f' % (env.now, name, wait))

def workload(env, number, inter_arrival_time, server):
    for i in range(number):
        req = request(env, 'Request%03d' % i, server, service_time=SVR_ST)
        env.process(req)
        t = random.expovariate(1.0 / inter_arrival_time)
        yield env.timeout(t)


开始仿真

In [34]:
random.seed(RANDOM_SEED)
env = simpy.Environment()

server = simpy.Resource(env, capacity=1)
env.process(workload(env, REQ_TOTAL, REQ_IAT, server))
env.run()


 0.0000 Request000: incoming
 0.0000 Request000: waited  0.000
 0.1773 Request000: completed  0.177
 8.1605 Request001: incoming
 8.1605 Request001: waited  0.000
 9.9286 Request001: completed  1.768
10.7335 Request002: incoming
10.7335 Request002: waited  0.000
18.6377 Request002: completed  7.904
21.4022 Request003: incoming
21.4022 Request003: waited  0.000
22.0389 Request003: completed  0.637
39.2205 Request004: incoming
39.2205 Request004: waited  0.000
39.4323 Request004: completed  0.212
43.6049 Request005: incoming
43.6049 Request005: waited  0.000
45.5786 Request006: incoming
45.5786 Request006: waited  0.000
45.7938 Request007: incoming
45.7938 Request007: waited  0.000
47.1305 Request006: completed  1.552
48.5323 Request005: completed  4.927
51.3051 Request007: completed  5.511
54.1897 Request008: incoming
54.1897 Request008: waited  0.000
56.1819 Request009: incoming
56.1819 Request009: waited  0.000
56.2276 Request009: completed  0.046
60.4184 Request008: completed  6.229


记录延迟数据

In [35]:
with open("latency.csv", "w+") as tracefile:
    tracefile.write("latency\n")
    tracefile.writelines([str(l) + '\n' for l in latency])